In [2]:
from IPython.display import IFrame
from IPython.core.display import display, HTML
import IPython
#display(HTML("<style>.container { width:90% !important; }</style>"))

In [ ]:
def show_app(app,  # type: dash.Dash
             port=9999,
             width=700,
             height=350,
             offline=True,
             style=True,
             **dash_flask_kwargs):
    """
    Run the application inside a Jupyter notebook and show an iframe with it
    :param app:
    :param port:
    :param width:
    :param height:
    :param offline:
    :return:
    """
    url = 'http://localhost:%d' % port
    iframe = '<iframe src="{url}" width={width} height={height}></iframe>'.format(url=url,
                                                                                  width=width,
                                                                                  height=height)
    IPython.display.display_html(iframe, raw=True)
    if offline:
        app.css.config.serve_locally = True
        app.scripts.config.serve_locally = True
    if style:
        external_css = ["https://fonts.googleapis.com/css?family=Raleway:400,300,600",
                        "https://maxcdn.bootstrapcdn.com/font-awesome/4.7.0/css/font-awesome.min.css",
                        "http://getbootstrap.com/dist/css/bootstrap.min.css", ]

        for css in external_css:
            app.css.append_css({"external_url": css})

        external_js = ["https://code.jquery.com/jquery-3.2.1.min.js",
                       "https://cdn.rawgit.com/plotly/dash-app-stylesheets/a3401de132a6d0b652ba11548736b1d1e80aa10d/dash-goldman-sachs-report-js.js",
                       "http://getbootstrap.com/dist/js/bootstrap.min.js"]

        for js in external_js:
            app.scripts.append_script({"external_url": js})

    return app.run_server(debug=False,  # needs to be false in Jupyter
                          port=port,
                          **dash_flask_kwargs)

In [3]:
from IPython.display import IFrame
IFrame('http://localhost:8050', width=1000, height=700)

In [ ]:
import time
import random
from context import cirrus
from cirrus import LogisticRegression
from cirrus import app

def progress_callback(time_loss, cost, task):
    print("Current training loss:", time_loss, "current cost ($): ", cost)
    
def progress_callback2(time_loss, cost, task):
    print("Current training loss:", time_loss, "current cost ($): ", cost)

ps_servers = [
    ('ec2-34-209-127-25.us-west-2.compute.amazonaws.com', '172.31.8.233', 0.0001),
    ('ec2-34-212-176-10.us-west-2.compute.amazonaws.com', '172.31.4.18', 0.1)
]

data_bucket = 'cirrus-criteo-kaggle-19b-random'
model = 'model_v1'

basic_params = {
    'n_workers': 4,
    'n_ps': 2,
    'worker_size': 128,
    'dataset': data_bucket,
    'learning_rate': 0.01,
    'epsilon': 0.0001,
    'progress_callback': progress_callback,
    'timeout': 0,
    'threshold_loss': 0,
    'resume_model': model,
    'key_name': 'mykey',
    'key_path': '/home/camus/Downloads/mykey.pem',
    'ps_ip_public': 'ec2-54-71-177-228.us-west-2.compute.amazonaws.com',
    'ps_ip_private': '172.31.9.205',
    'ps_username': 'ubuntu',
    'opt_method': 'adagrad',
    'checkpoint_model': 60,
    'minibatch_size': 20,
    'model_bits': 19,
    'use_grad_threshold': False,
    'grad_threshold': 0.001,
    'train_set': (0,824),
    'test_set': (835,840)
}

if __name__ == "__main__":
    batch = []
    index = 0
    for ps in ps_servers:
        config = basic_params.copy()
        config['ps_ip_public'] = ps[0]
        config['ps_ip_private'] = ps[1]
        config['learning_rate'] = ps[2]
        cirrus_obj = LogisticRegression(**config)
        cirrus_obj.id = index
        cirrus_obj.run()
        batch.append(cirrus_obj)
        index += 1
    app.bundle = batch
    print("Bootstrapping")
    app.app.run_server(debug=False)
    #time.sleep(10)
    #batch[0].kill()
    #batch[1].kill()


Running Logistic Regression workload
Starting LogisticRegressionTask
User's specific ip: ec2-34-209-127-25.us-west-2.compute.amazonaws.com
Copying ps to vm..
Done waiting... Attempting to copy over binary
Copied PS binary to VM
Defining configuration file

input_path: /mnt/efs/criteo_kaggle/train.csv 
input_type: csv
num_classes: 2 
num_features: 13 
limit_cols: 14 
normalize: 1 
limit_samples: 50000000 
s3_size: 50000 
use_bias: 1 
model_type: LogisticRegression 
minibatch_size: 20 
learning_rate: 0.000100 
epsilon: 0.000100 
model_bits: 19 
s3_bucket: cirrus-criteo-kaggle-19b-random 
use_grad_threshold: 0 
grad_threshold: 0.001000 
train_set: 0-824 
test_set: 835-840

Launching ps
Launching parameter server
('cmd:', 'ssh -o "StrictHostKeyChecking no" -i /home/camus/Downloads/mykey.pem ubuntu@ec2-34-209-127-25.us-west-2.compute.amazonaws.com "nohup ./parameter_server --config config.txt --nworkers 10000 --rank 1 &> ps_output &"')
Launching lambdas
Starting error task
 ('cmd', 'ssh -o 

 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)


Launching lambdas
Lambdas have been launchedStarting error task
Bootstrapping

('cmd', 'ssh -o "StrictHostKeyChecking no" -i /home/camus/Downloads/mykey.pem ubuntu@ec2-34-212-176-10.us-west-2.compute.amazonaws.com "./parameter_server --config config.txt --nworkers 10 --rank 2 --ps_ip "172.31.4.18"" > error_out_1 &')
Cost Model
PS has 3 lambdas and 
0.0001
Launching more lambdas
payload: {"num_task": 3, "num_workers": 4, "ps_ip": "172.31.8.233"}
PS has 0 lambdas and 
0.1
Launching more lambdas
payload: {"num_task": 3, "num_workers": 4, "ps_ip": "172.31.4.18"}


127.0.0.1 - - [29/Jun/2018 15:12:58] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Jun/2018 15:12:59] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [29/Jun/2018 15:12:59] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [29/Jun/2018 15:12:59] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [29/Jun/2018 15:12:59] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [29/Jun/2018 15:12:59] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [29/Jun/2018 15:12:59] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [29/Jun/2018 15:12:59] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [29/Jun/2018 15:12:59] "POST /_dash-update-component HTTP/1.1" 200 -


('Get time loss', 0)
('Get time loss', 0)
('Get time loss', 1)
('Get time loss', 1)
all
killall triggered!


...

('Get time loss', 0)
('Get time loss', 0)
('Get time loss', 1)
('Get time loss', 1)
all


127.0.0.1 - - [29/Jun/2018 15:37:30] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [29/Jun/2018 15:37:30] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [29/Jun/2018 15:37:30] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [29/Jun/2018 15:37:30] "POST /_dash-update-component HTTP/1.1" 200 -


('Current training loss:', (1471.2, 0.625024), 'current cost ($): ', 0.09095910851298014)
('1471.2', '0.625024', 0.1)
('Get time loss', 0)
('Get time loss', 0)
('Get time loss', 1)
('Get time loss', 1)
all
PS has 5 lambdas and 
0.1


127.0.0.1 - - [29/Jun/2018 15:37:31] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [29/Jun/2018 15:37:31] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [29/Jun/2018 15:37:31] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [29/Jun/2018 15:37:31] "POST /_dash-update-component HTTP/1.1" 200 -


('Get time loss', 0)
('Get time loss', 0)
('Get time loss', 1)
('Get time loss', 1)
all


In [ ]:
#[a.run() for a in batch]

#show_app(app.app)